In [2]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from pathlib import Path
import os
from plotnine import *
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from descartes import PolygonPatch
import alphashape
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import Point
import math
from scipy.stats import kde
import hdbscan
from sklearn.cluster import DBSCAN
from scipy.spatial import Delaunay
from collections import defaultdict
from itertools import permutations
import plotly.figure_factory as ff
import matplotlib.cm as cm
import itertools
from scipy.stats import wilcoxon #Wilcoxon is nonparametric, paired data = paired student T test ***
from scipy.stats import pearsonr

In [3]:
# Number of neighbors in different types of cluster
# Load cluster-participant cell composition
df_Cluster_pv2 = pd.read_csv("C:/Data/DCs_results/c_Type.txt")
df_Cluster_pv2_Pt = df_Cluster_pv2[~df_Cluster_pv2["Pt"].isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])].drop(columns=["Unnamed: 0"])
# Load cluster-overall size (include all cells)
SIZE_cType = pd.read_csv("C:/Data/DCs_results/SIZE_c_Type.txt")
SIZE_cType_Pt = SIZE_cType[~SIZE_cType["Pt"].isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])].drop(columns=["Unnamed: 0"])
# Load cluster-overall size (include all cells, with mean in each specimen)
SIZE_cType_Pts = pd.read_csv("C:/Data/DCs_results/SIZE_cType_Pts.txt")
# Load neighbor-cluster data, remove control samples (each cell, numbers of neighbors, participated cluster ID)
all_files_cType = pd.read_csv("C:/Data/DCs_results/nn_DCs_clustera_cType.txt")
all_files_cType_Pt = all_files_cType[~all_files_cType["Pt"].isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])].drop(columns=["Unnamed: 0", "Unnamed: 0.1"]).dropna(
    subset=["X", "Y"]).fillna(0)

# Hetero-cDC1
neighbors_for_cDC1_Het = all_files_cType_Pt[(all_files_cType_Pt.c_Type == "Het")&(all_files_cType_Pt.Phenotype.isin(["CD141+PD-L1-","CD141+PD-L1+"]))]
neighbors_for_cDC1_Het["With Total CD1a+ Neighbors"] = neighbors_for_cDC1_Het["CD1a+"] + neighbors_for_cDC1_Het["CD1a+PD-L1+"] + neighbors_for_cDC1_Het["CD1a+PD-L1-"]
neighbors_for_cDC1_Het_sp = neighbors_for_cDC1_Het[["Pt","Phenotype","With Total CD1a+ Neighbors"]]
neighbors_for_cDC1_Het_sp2 = neighbors_for_cDC1_Het_sp.pivot_table(index=["Pt","With Total CD1a+ Neighbors"],columns=["Phenotype"],
aggfunc=lambda x: len(x)).fillna(0)
neighbors_for_cDC1_Het_sp2.reset_index(inplace=True)
neighbors_for_cDC1_Het_sp2["%PD-L1+"] = 100*neighbors_for_cDC1_Het_sp2["CD141+PD-L1+"]/(neighbors_for_cDC1_Het_sp2["CD141+PD-L1+"]+neighbors_for_cDC1_Het_sp2["CD141+PD-L1-"])
neighbors_for_cDC1_Het_sp3 =neighbors_for_cDC1_Het_sp2.drop(columns=["CD141+PD-L1+","CD141+PD-L1-"]).round(0)
neighbors_for_cDC1_Het_sp3 = neighbors_for_cDC1_Het_sp3[~neighbors_for_cDC1_Het_sp3["With Total CD1a+ Neighbors"].isin([8, 9])]
neighbors_for_cDC1_Het_sp3["With Total CD1a+ Neighbors"]=neighbors_for_cDC1_Het_sp3["With Total CD1a+ Neighbors"].astype(int)
# Plot Hetero-cDC1
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g= sns.catplot(data = neighbors_for_cDC1_Het_sp3, kind="bar", height = 6, aspect = .75, 
               x="With Total CD1a+ Neighbors", y= "%PD-L1+", edgecolor="0.1", color="plum")
plt.xlabel('Number of CD1a+ neighbor cell', fontsize=17)
plt.ylabel('%PD-L1+ on cDC1 cells',fontsize=17)
plt.ylim((0,100))
g.savefig('C:/Data/DCs_neighbors/%PD-L1 on cDC1_NoCD1A(Hetero).tiff', format='tiff', dpi=600)
plt.show()

# Iso-cDC1
neighbors_for_cDC1_Iso = all_files_cType_Pt[(all_files_cType_Pt.c_Type == "iso")&(all_files_cType_Pt.Phenotype.isin(["CD141+PD-L1-","CD141+PD-L1+"]))]
neighbors_for_cDC1_Iso["With Total CD1a+ Neighbors"] = neighbors_for_cDC1_Iso["CD1a+"] + neighbors_for_cDC1_Iso["CD1a+PD-L1+"] + neighbors_for_cDC1_Iso["CD1a+PD-L1-"]
neighbors_for_cDC1_Iso_sp = neighbors_for_cDC1_Iso[["Pt","Phenotype","With Total CD1a+ Neighbors"]]
neighbors_for_cDC1_Iso_sp2 = neighbors_for_cDC1_Iso_sp.pivot_table(index=["Pt","With Total CD1a+ Neighbors"],columns=["Phenotype"],
aggfunc=lambda x: len(x)).fillna(0)
neighbors_for_cDC1_Iso_sp2.reset_index(inplace=True)
neighbors_for_cDC1_Iso_sp2["%PD-L1+"] = 100*neighbors_for_cDC1_Iso_sp2["CD141+PD-L1+"]/(neighbors_for_cDC1_Iso_sp2["CD141+PD-L1+"]+neighbors_for_cDC1_Iso_sp2["CD141+PD-L1-"])
neighbors_for_cDC1_Iso_sp3 =neighbors_for_cDC1_Iso_sp2.drop(columns=["CD141+PD-L1+","CD141+PD-L1-"]).round(0)
neighbors_for_cDC1_Iso_sp3 = neighbors_for_cDC1_Iso_sp3[~neighbors_for_cDC1_Iso_sp3["With Total CD1a+ Neighbors"].isin([3])]
neighbors_for_cDC1_Iso_sp3["With Total CD1a+ Neighbors"]=neighbors_for_cDC1_Iso_sp3["With Total CD1a+ Neighbors"].astype(int)
# Plot Iso-cDC1
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g= sns.catplot(data = neighbors_for_cDC1_Iso_sp3, kind="bar", height = 6, aspect = .5, 
               x="With Total CD1a+ Neighbors", y= "%PD-L1+", edgecolor="0.1", color="plum")
plt.xlabel('', fontsize=17)
plt.ylabel('%PD-L1+ on cDC1 cells',fontsize=17)
plt.ylim((0,100))
g.savefig('C:/Data/DCs_neighbors/%PD-L1 on cDC1_NoCD1A(Iso).tiff', format='tiff', dpi=600)
plt.show()

# Hetero-CD1A
neighbors_for_CD1A_Het = all_files_cType_Pt[(all_files_cType_Pt.c_Type == "Het")&(all_files_cType_Pt.Phenotype.isin(["CD1a+PD-L1-","CD1a+PD-L1+","CD1a+"]))]
neighbors_for_CD1A_Het["With Total cDC1 Neighbors"] = neighbors_for_CD1A_Het["CD141+PD-L1-"] + neighbors_for_CD1A_Het["CD141+PD-L1+"]
neighbors_for_CD1A_Het_sp = neighbors_for_CD1A_Het[["Pt","Phenotype","With Total cDC1 Neighbors"]]
neighbors_for_CD1A_Het_sp2 = neighbors_for_CD1A_Het_sp.pivot_table(index=["Pt","With Total cDC1 Neighbors"],columns=["Phenotype"],
aggfunc=lambda x: len(x)).fillna(0)
neighbors_for_CD1A_Het_sp2.reset_index(inplace=True)
neighbors_for_CD1A_Het_sp2["%PD-L1+"] = 100*neighbors_for_CD1A_Het_sp2["CD1a+PD-L1+"]/(neighbors_for_CD1A_Het_sp2["CD1a+PD-L1+"]+neighbors_for_CD1A_Het_sp2["CD1a+PD-L1-"])
neighbors_for_CD1A_Het_sp3 =neighbors_for_CD1A_Het_sp2.drop(columns=["CD1a+","CD1a+PD-L1+","CD1a+PD-L1-"]).round(0)
neighbors_for_CD1A_Het_sp3["With Total cDC1 Neighbors"]=neighbors_for_CD1A_Het_sp3["With Total cDC1 Neighbors"].astype(int)

# Plot Hetero-CD1A
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g= sns.catplot(data = neighbors_for_CD1A_Het_sp3, kind="bar", height = 6, aspect = .75, 
               x="With Total cDC1 Neighbors", y= "%PD-L1+", edgecolor="0.1", color="plum")
plt.xlabel('Number of cDC1 neighbor cell', fontsize=17)
plt.ylabel('%PD-L1+ on CD1a+ cells',fontsize=17)
plt.ylim((0,100))
g.savefig('C:/Data/DCs_neighbors/%PD-L1 on CD1A_NocDC1(Hetero).tiff', format='tiff', dpi=600)
plt.show()

# Iso-CD1A
neighbors_for_CD1A_Iso = all_files_cType_Pt[(all_files_cType_Pt.c_Type == "iso")&(all_files_cType_Pt.Phenotype.isin(["CD1a+PD-L1-","CD1a+PD-L1+","CD1a+"]))]
neighbors_for_CD1A_Iso["With Total cDC1 Neighbors"] = neighbors_for_CD1A_Iso["CD141+PD-L1-"] + neighbors_for_CD1A_Iso["CD141+PD-L1+"]
neighbors_for_CD1A_Iso_sp = neighbors_for_CD1A_Iso[["Pt","Phenotype","With Total cDC1 Neighbors"]]
neighbors_for_CD1A_Iso_sp2 = neighbors_for_CD1A_Iso_sp.pivot_table(index=["Pt","With Total cDC1 Neighbors"],columns=["Phenotype"],
aggfunc=lambda x: len(x)).fillna(0)
neighbors_for_CD1A_Iso_sp2.reset_index(inplace=True)
neighbors_for_CD1A_Iso_sp2["%PD-L1+"] = 100*neighbors_for_CD1A_Iso_sp2["CD1a+PD-L1+"]/(neighbors_for_CD1A_Iso_sp2["CD1a+PD-L1+"]+neighbors_for_CD1A_Iso_sp2["CD1a+PD-L1-"])
neighbors_for_CD1A_Iso_sp3 =neighbors_for_CD1A_Iso_sp2.drop(columns=["CD1a+","CD1a+PD-L1+","CD1a+PD-L1-"]).round(0)
neighbors_for_CD1A_Iso_sp3 = neighbors_for_CD1A_Iso_sp3[~neighbors_for_CD1A_Iso_sp3["With Total cDC1 Neighbors"].isin([4])]
neighbors_for_CD1A_Iso_sp3["With Total cDC1 Neighbors"]=neighbors_for_CD1A_Iso_sp3["With Total cDC1 Neighbors"].astype(int)

# Plot Iso-CD1A
sns.set(style = "ticks", font_scale = 1.5, color_codes=True)
sns.set_context("talk")
g= sns.catplot(data = neighbors_for_CD1A_Iso_sp3, kind="bar", height = 6, aspect = .55, 
               x="With Total cDC1 Neighbors", y= "%PD-L1+", edgecolor="0.1", color="plum")
#plt.xlabel('Number of cDC1 neighbor cell', fontsize=17)
plt.ylabel('%PD-L1+ on CD1a+ cells',fontsize=17)
plt.ylim((0,100))
g.savefig('C:/Data/DCs_neighbors/%PD-L1 on CD1A_NocDC1(Iso).tiff', format='tiff', dpi=600)
plt.show()